In [1]:
!pip install gekko

     |████████████████████████████████| 12.4 MB 3.8 MB/s eta 0:00:01


Here's a [link](https://apmonitor.com/do/index.php/Main/ModelFormulation) the this homework's webpage. Basically, we are simulated fluid heights over 4 interconnected lakes (i.e. tanks) in Utah. There are several ways to do this. I'll start by using GEKKO.

Here's a list of parameters for this problem:

```
 Outflow River Rates (km3/yr) with height in meters
 Vflow_out1 = 0.030 sqrt(h1) 
 Vflow_out2 = 0.015 sqrt(h2) 
 Vflow_out3 = 0.060 sqrt(h3)
 Vflow_out4 = 0

 Evaporation Rates (km3/yr)
 Vevap = 0.5e-5 * Area, for salt water (Great Salt Lake)
 Vevap = 1e-5 * Area, for fresh water (all others)

 Inflow Rates (km3/yr)
 Vflow_in1 = 0.13 (June-Feb), 0.21 (Mar-May)
 Vflow_in2 = Vflow_out1
 Vflow_in3 = Vflow_out2
 Vflow_in4 = Vflow_out3

 Usage Requirements (km3/yr)
 Vuse1 = 0.03
 Vuse2 = 0.05
 Vuse3 = 0.02
 Vuse4 = 0.00

 Area of Reservoir / Lake (km2)
 A1 = 13.4
 A2 = 12.0
 A3 = 384.5
 A4 = 4400

 Initial Volume of Reservoir / Lake (km3)
 V1 = 0.26
 V2 = 0.18
 V3 = 0.68
 V4 = 22.0
 ```

In general, this is going to be a system of 4 first-order coupled ODEs with 4 state variables representing the height of each of the reservoirs. 

We can start with a mass balance of a single reservoir (tank, since we are assuming areas are constant as well):

$$ \frac{dm}{dt} = \dot{m}_{\mathrm{in}} - \dot{m}_{\mathrm{out}} $$

Since $m = \rho V \implies \dot{m} = \rho \dot{V} = \rho A \dot{h}$ (assuming constant density of water), we can rewrite the above equation as:

$$ \rho A \frac{dh}{dt} = \rho \dot{V}_{\mathrm{in}} - \rho \dot{V}_{\mathrm{out}} $$

or 

$$ A \frac{dh}{dt} =  \dot{V}_{\mathrm{in}} -  \dot{V}_{\mathrm{out}} $$

The table above tells us that there is one source of volumtric flow for each tank and several sources of outflow for each tank (usage, outflow, and evaporation).

So the final ODE for a single tank is going to look something like this:

``` A * hdot = Vflow_in - k * sqrt(h) - c * A - Vuse ```


In [4]:
from gekko import GEKKO
import numpy as np

#constant parameters
areas = np.array([13.4,12.0,384.5,4400])

#outflow coefficients
c = np.array([.03, .015,.06, 0])

#initial conditions
V0 = np.array([.26,.18,.68,22])
h0 = 1000 * V0 /areas

#initial outflow rates
Vout0 = c * np.sqrt(h0)

#time varying inflow rate into first tank
vin_first = np.ones(12)*.13
#except during spring with snow melting
vin_first[2:4] = .21

Vin = np.zeros(4)


In [5]:
#GEKKO STUFF

m = GEKKO()
m.time = np.arange(0,12,1)